# CS337 - OPERATING SYSTEMS- Project 7 Software Synchronization
#### By: Matthew Bass


### Table of Contents

* [Overview](#over)
* [Setup](#set)
* [Conclusion](#con)



**Project Overview:** <a class="anchor" id="over"></a>

In this project I will, write a small class to implement a `Semaphore` class
using a `counter` and a `condition` variable.  Here’s the documentation for
[condition variables/objects in Python](https://docs.python.org/3/library/threading.html#condition-objects)



### PREREQUISITES:
    Python 3
    Jupyter
    jupyter_contrib_nbextensions
    random
    threading
    datclasses
    autopep8


<br>

-----

### Setup: <a class="anchor" id="set"></a>

To start I am using this code to make all my code blocks hide-able, thanks to
[stackoverflow](https://stackoverflow.com/questions/27934885/how-to-hide-code-from-cells-in-ipython-notebook-visualized-with-nbviewer).



In [5]:
import threading
%%HTML
<script>
    function luc21893_refresh_cell(cell) {
        if( cell.luc21893 ) return;
        cell.luc21893 = true;
        console.debug('New code cell found...' );

        var div = document.createElement('DIV');
        cell.parentNode.insertBefore( div, cell.nextSibling );
        div.style.textAlign = 'right';
        var a = document.createElement('A');
        div.appendChild(a);
        a.href='#'
        a.luc21893 = cell;
        a.setAttribute( 'onclick', "luc21893_toggle(this); return false;" );

        cell.style.visibility='hidden';
        cell.style.position='absolute';
        a.innerHTML = '[show code]';

    }
    function luc21893_refresh() {
        if( document.querySelector('.code_cell .input') == null ) {
            // it apeears that I am in a exported html
            // hide this code
            var codeCells = document.querySelectorAll('.jp-InputArea')
            codeCells[0].style.visibility = 'hidden';
            codeCells[0].style.position = 'absolute';
            for( var i = 1; i < codeCells.length; i++ ) {
                luc21893_refresh_cell(codeCells[i].parentNode)
            }
            window.onload = luc21893_refresh;
        }
        else {
            // it apperas that I am in a jupyter editor
            var codeCells = document.querySelectorAll('.code_cell .input')
            for( var i = 0; i < codeCells.length; i++ ) {
                luc21893_refresh_cell(codeCells[i])
            }
            window.setTimeout( luc21893_refresh, 1000 )
        }
    }

    function luc21893_toggle(a) {
        if( a.luc21893.style.visibility=='hidden' ) {
            a.luc21893.style.visibility='visible';
            a.luc21893.style.position='';
            a.innerHTML = '[hide code]';
        }
        else {
            a.luc21893.style.visibility='hidden';
            a.luc21893.style.position='absolute';
            a.innerHTML = '[show code]';
        }
    }

    luc21893_refresh()
</script>

SyntaxError: invalid syntax (699355145.py, line 3)

#### Imports:

In [7]:
import threading
import random
import time
from dataclasses import dataclass, field



### Semaphore Class:

In [8]:
@dataclass
class Semaphore():
    '''
    A Semaphore class that allows for the use of a counter and a condition
    variable.

    Attributes:
        counter: An integer that represents the number of threads that are
            currently waiting on the condition variable.
        condition: A condition variable that is used to signal threads that
            the counter is 0.
    '''
    counter: int = field(default=1)
    condition: threading.Condition = field(default = threading.Condition(),
                                           init=False)

    def acquire(self):
        '''
        This method acquires the lock for the condition variable before
        decrementing the counter by one, then it checks if the counter is
        below zero and sets the thread to sleep if true.  Otherwise, it
        releases the lock.

        '''

        # acquire the lock
        self.condition.acquire()

        # decrement the counter
        self.counter -= 1

        # check if the counter is below zero
        # if so, set the thread to sleep
        # otherwise, release the lock
        if self.counter < 0:
            self.condition.wait()
        else:
            self.condition.release()

        return




    def release(self):
        '''
        This method acquires the condition lock and increments the counter by
        one, notifies a single sleeping thread, and releases the lock.
        '''

        # acquire the lock
        self.condition.acquire()

        # increment the counter
        self.counter += 1

        # notify a single sleeping thread
        self.condition.notify()

        # release the lock
        self.condition.release()

        return


##### Test Semaphore Class:

Now to test my `Semaphore` class, I will use the Buffer example from class where
 we used Python semaphores and replace the Python semaphores with my own
 `Semaphore` class. The output should be similar to the python semaphore
 solution, in the sense that:

1. We see no errors, especially pop with empty list error.

2. The buffer never exceeds 10.

3. Every item added to the buffer is removed at the end of each run.


This buffer simulation is a producer and consumer problem.  The producers
will put items into the `BUFFER`, and the consumers will take items out of the
`BUFFER`.  The buffer is bounded by a maximum size of 10.  The items will be
unique numbers 1 - `BUFFER_SIZE`.

There will be 3 `Semaphores` used in this simulation.  The first is
`ACCESS` which is used to synchronize access to the `BUFFER` between the
consumer and producer threads, it starts with a value of 1 for the counter
and is decremented by 1 for each thread that wants to access the buffer. The
second is `EMPTY` which is used to signal the number of empty spaces in the
`BUFFER` ut starts with a value the size of `BUFFER_SIZE` and is decremented
by 1 for each thread that adds an item to the `BUFFER`. The third is `FULL`
which is used to signal the number of full spaces in the `BUFFER` and starts
at 0 and is incremented by 1 for each thread that adds an item to the `BUFFER`.

Here are the shared global variables:

In [9]:
# Buffer Variables
BUFFER_SIZE = 10
BUFFER = []

# The Number of Items each producer will put into the buffer
NUM_PRODUCE = 10

# The number of producer and consumer threads
NUM_PRODUCERS = 1
NUM_CONSUMERS = 1

# Semaphores
ACCESS = Semaphore(counter=1)
EMPTY = Semaphore(counter=BUFFER_SIZE)
FULL = Semaphore(counter=0)


To start I will make the producer and consumer threads.

`producerFunc()`:  Will put a unique number into the buffer

In [10]:
def producerFunc(debug: bool = True):
    '''
    This function will put a unique number into the buffer.

    args:
        debug: A boolean that will print debug statements if true.
    '''

    # Get thr global variables
    global BUFFER, ACCESS, EMPTY, FULL

    for _ in range(NUM_PRODUCE):

        # Produce a unique number
        data = random.randint(1, 100000000)

        # Acquire the empty semaphore
        EMPTY.acquire()

        # Acquire the access semaphore
        ACCESS.acquire()

        # Add the data to the buffer
        BUFFER.append(data)

        if debug:
            print(f'Producer {threading.current_thread().name} added {data} to the buffer')

        # Release the access semaphore
        ACCESS.release()

        # Release the full semaphore
        FULL.release()

        # Sleep for a random amount of time
        time.sleep(1)

    return

`consumerFunc`:  Will take a unique number from the buffer.

In [11]:
def consumerFunc(debug: bool = True):
    '''
    This function will put a unique number into the buffer.

    Args:
        debug:  If true, will print debug statements.
    '''

    # Get thr global variables
    global BUFFER, ACCESS, EMPTY, FULL

    for _ in range(NUM_PRODUCE):



        # Acquire the full semaphore
        FULL.acquire()

        # Acquire the access semaphore
        ACCESS.acquire()

        # get the data from the buffer
        data = BUFFER.pop()

        if debug:
            print(f'Producer {threading.current_thread().name} poped {data} '
                  f'from the buffer')


        # Release the access semaphore
        ACCESS.release()

        # Release the empty semaphore
        EMPTY.release()

        # Sleep for a random amount of time
        time.sleep(2.5)

    return


`bufferSimulation()`:  Will run the producer and consumer threads and the
buffer simulation to test the `Semaphore` class.

In [16]:
def bufferSimulation():

    # Create the producer and consumer threads
    producer = threading.Thread(target=producerFunc)
    consumer = threading.Thread(target=consumerFunc)

    # Start the producer and consumer threads
    producer.start()
    consumer.start()

    # Wait for the producer and consumer threads to finish
    producer.join()
    consumer.join()

    if len(BUFFER) == 0:
        print('The buffer is empty')
    else:
        print('The buffer is not empty')

    print(f'Done with the simulation')

Now I will run the buffer simulation.

In [17]:
bufferSimulation()

Producer Thread-12 added 55619276 to the buffer
Producer Thread-13 poped 55619276 from the buffer
Producer Thread-12 added 35736040 to the buffer
Producer Thread-12 added 6602560 to the buffer
Producer Thread-13 poped 6602560 from the buffer
Producer Thread-12 added 86056417 to the buffer
Producer Thread-12 added 96901518 to the buffer
Producer Thread-13 poped 96901518 from the buffer
Producer Thread-12 added 41333785 to the buffer
Producer Thread-12 added 5985016 to the buffer
Producer Thread-12 added 4282006 to the buffer
Producer Thread-13 poped 4282006 from the buffer
Producer Thread-12 added 50685753 to the buffer
Producer Thread-12 added 34683031 to the buffer
Producer Thread-13 poped 34683031 from the buffer
Producer Thread-13 poped 50685753 from the buffer
Producer Thread-13 poped 5985016 from the buffer
Producer Thread-13 poped 41333785 from the buffer
Producer Thread-13 poped 86056417 from the buffer
Producer Thread-13 poped 35736040 from the buffer
The buffer is empty
Done w

<br>

### Dining Philosophers Problem:

In [ ]:
def philosopherFunc(thread_id: int):

In [17]:
def diningPhilosophers(num_threads: int = 5, debug: bool = True):
    '''
    This function will simulate the dining philosophers problem.

    Args:
        num_threads:  The number of threads to create.
        debug:  If true, will print debug statements.
    '''

    # Get the global variables
    global ACCESS, LEFT, RIGHT, FULL, EMPTY

    # Create the threads
    threads = []
    for _ in range(num_threads):
        threads.append(threading.Thread(target=philosopherFunc))

    # Start the threads
    for thread in threads:
        thread.start()

    # Wait for the threads to finish
    for thread in threads:
        thread.join()

    if debug:
        print(f'Done with the simulation')


<br>


### Conclusion:

Overall, I found this project



#### Resources:
- [Dr. Al Madi](https://www.cs.colby.edu/nsalmadi/)
- [Using With Statements User Defined Objects](https://en.wikipedia.org/wiki/Peterson%27s_algorithm#Filter_algorithm:_Peterson's_algorithm_for_more_than_two_processes)